In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

# load_summarize_chain

In [2]:
import json
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI
from langchain.callbacks.base import BaseCallbackHandler

# 웹 기반 문서 로더를 초기화합니다.
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")

# 문서를 로드합니다.
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs):
        print(f"{token}", end="", flush=True)

In [4]:
# OpenAI의 Chat 모델을 초기화합니다. 여기서는 온도를 0으로 설정하고 모델 이름을 지정합니다.
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o-mini",
    streaming=True,
    callbacks=[StreamCallback()],
)

In [5]:
# 요약 체인을 로드합니다. 체인 타입을 'stuff'로 지정합니다.
chain = load_summarize_chain(llm, chain_type="stuff")

In [6]:
# 문서에 대해 요약 체인을 실행합니다.
answer = chain.invoke({"input_documents": docs})
print(answer["output_text"])

The article "LLM Powered Autonomous Agents" by Lilian Weng explores the development and capabilities of autonomous agents powered by large language models (LLMs). It outlines a system architecture comprising three main components: 

1. **Planning**: Agents decompose complex tasks into manageable subgoals and engage in self-reflection to improve future actions.
2. **Memory**: The system utilizes short-term and long-term memory, with techniques like Maximum Inner Product Search (MIPS) for efficient information retrieval.
3. **Tool Use**: Agents can leverage external APIs and tools to enhance their functionality, demonstrated through various case studies, including scientific discovery and generative simulations.

The article also discusses challenges such as finite context length, difficulties in long-term planning, and the reliability of natural language interfaces. It highlights several proof-of-concept projects like AutoGPT and GPT-Engineer, showcasing the potential and limitations of

In [8]:
# 요약 결과를 JSON 형식으로 저장합니다.
output_data = {
    "summary": answer["output_text"]
}
# 결과를 JSON 파일로 저장합니다.
with open("load_summarize_chain_result.json", "w", encoding="utf-8") as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

print("JSON 파일로 저장되었습니다.")

JSON 파일로 저장되었습니다.


# 1. Stuff
##### 문서 전체를 프롬프트에 넣는 방식. 짧은 문서인 경우 사용

In [9]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain import hub

In [10]:
# 요약문을 작성하기 위한 프롬프트 정의 (직접 프롬프트를 작성하는 경우)
prompt_template = """Please summarize the sentence in 5 sentences according to the following REQUEST.
REQUEST:
1. Summarize the main points in bullet points.
2. DO NOT translate any technical terms.
3. DO NOT include any unnecessary information.
CONTEXT:
{context}

SUMMARY:"
"""
prompt = PromptTemplate.from_template(prompt_template)

# # 원격 저장소에서 프롬프트를 가져오는 경우
# prompt = hub.pull("teddynote/summary-stuff-documents-korean")

In [11]:
# LLM 체인 정의
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o-mini",
    streaming=True,
    callbacks=[StreamCallback()],
)

In [12]:
# LLMChain 정의
llm_chain = LLMChain(llm=llm, prompt=prompt)

/tmp/ipykernel_1081/3440632591.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)


In [13]:
# StuffDocumentsChain 정의
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="context")

/tmp/ipykernel_1081/607598605.py:2: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain/
  stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="context")


In [14]:
docs = loader.load()
response = stuff_chain.invoke({"input_documents": docs})

- LLM-powered autonomous agents utilize large language models as their core controllers, enhancing their capabilities in planning, memory, and tool use.
- Key components include task decomposition for efficient handling of complex tasks, short-term and long-term memory for information retention, and the ability to call external APIs for additional information.
- Challenges faced by these agents include limited context length, difficulties in long-term planning, and reliability issues with natural language interfaces.
- Various proof-of-concept examples, such as AutoGPT and GPT-Engineer, demonstrate the potential and limitations of LLMs in autonomous task execution.
- The integration of memory and reflection mechanisms allows agents to learn from past actions, improving their performance over time.

In [18]:
# 요약 결과를 JSON 형식으로 저장합니다.
output_data = {
    "prompt": prompt_template,
    "summary": answer["output_text"]
}
# 결과를 JSON 파일로 저장합니다.
with open("Stuff_result.json", "w", encoding="utf-8") as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

print("JSON 파일로 저장되었습니다.")

JSON 파일로 저장되었습니다.


# 2. Map-Reduce
##### N개의 Chunk로 분할하고, 각 Chunk를 각각 요약한 후, 요약 내용을 합해서 다시 요약하는 방식.
##### 각 Chunk를 요약할 때 병렬로 요약할 수 있기 때문에 Refine 방법보다 빠르지만 Reduce 과정 포함하므로 API 호출 횟수가 더 많아짐 

In [19]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import hub

In [20]:
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o-mini",
    streaming=True,
    callbacks=[StreamCallback()],
)

In [30]:
# Map Prompt를 정의합니다.
map_template = """The following is a news article:
{docs}
Summarize the key facts, causes, and consequences of the events in the article in 3 sentences or fewer. Focus on the most important people, organizations, and facts, and exclude unnecessary details.
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)

# langchain 허브에서 'rlm/map-prompt'를 가져옵니다.
# map_prompt = hub.pull("teddynote/map-prompt")
map_prompt

PromptTemplate(input_variables=['docs'], input_types={}, partial_variables={}, template='The following is a news article:\n{docs}\nSummarize the key facts, causes, and consequences of the events in the article in 3 sentences or fewer. Focus on the most important people, organizations, and facts, and exclude unnecessary details.\nHelpful Answer:')

In [31]:
# LLMChain 인스턴스를 생성하며, 이때 LLM과 프롬프트로 'map_prompt'를 사용합니다.
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [32]:
# Reduce Prompt를 정의합니다.
reduce_template = """The following is a set of partial summaries of a news article:
{docs}
Please combine these partial summaries into a final summary of 8 sentences or fewer. Focus on synthesizing the key points and avoid repetition.
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
# prompt hub에서도 얻을 수 있음을 위에서 언급했듯이
# reduce_prompt = hub.pull("teddynote/reduce-prompt-korean")
reduce_prompt

PromptTemplate(input_variables=['docs'], input_types={}, partial_variables={}, template='The following is a set of partial summaries of a news article:\n{docs}\nPlease combine these partial summaries into a final summary of 8 sentences or fewer. Focus on synthesizing the key points and avoid repetition.\nHelpful Answer:')

In [33]:
# input_variables 없이 LLMChain 생성
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# 문서 리스트를 받아 하나의 문자열로 결합한 후 LLMChain에 전달
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# 매핑된 문서들을 결합하고 반복적으로 축소
reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_documents_chain,
    collapse_documents_chain=combine_documents_chain,
    token_max=4096,
)


In [34]:
# 문서들을 매핑하여 체인을 거친 후 결과를 결합하는 과정
map_reduce_chain = MapReduceDocumentsChain(
    # 매핑 체인
    llm_chain=map_chain,
    # 리듀스 체인
    reduce_documents_chain=reduce_documents_chain,
    # llm_chain에서 문서들을 넣을 변수 이름
    document_variable_name="docs",
    # 매핑 단계의 결과를 출력에 포함시킴
    return_intermediate_steps=False,
)

In [35]:
# 문자를 기준으로 텍스트를 분할하는 객체 생성
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=len,
)

# 문서들을 분할
split_docs = text_splitter.split_documents(docs)

<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_1081/3743367926.py:5: SyntaxWarning: invalid escape sequence '\.'
  separators=["\n\n", "\n", "(?<=\. )", " ", ""],


In [36]:
# split_docs를 map_reduce_chain의 run 메서드에 전달하여 실행한 결과를 출력합니다.
summary_result = map_reduce_chain.invoke({"input_documents": split_docs})

The article discusses the development of LLM (Large Language Model) powered autonomous agents, highlighting their components such as planning, memory, and tool use, along with case studies demonstrating their applications in scientific discovery and simulations. It emphasizes the challenges faced in creating these agents and the potential impact they could have on various fields. The author, Lilian Weng, provides insights into the mechanisms that enable these agents to function effectively and the implications of their use.The article discusses the potential of large language models (LLMs) as the core controllers for autonomous agent systems, highlighting examples like AutoGPT, GPT-Engineer, and BabyAGI. These systems utilize LLMs for planning by breaking down tasks into subgoals and enabling self-reflection to improve future performance. The overarching theme is the transformative capability of LLMs in solving complex problems beyond traditional text generation.The article discusses t

In [37]:
print(summary_result["output_text"])

The article examines recent advancements in Large Language Model (LLM) powered autonomous agents, emphasizing their components such as planning, memory, and tool use. It highlights frameworks like Chain of Thought (CoT), Tree of Thoughts, and ReAct, which enhance reasoning and decision-making capabilities. The integration of short-term and long-term memory types is crucial for improving agent performance, while external tools and APIs, exemplified by frameworks like API-Bank and ChemCrow, augment LLMs for specialized tasks. Despite these advancements, challenges such as hallucination, context understanding, and adversarial vulnerabilities persist, necessitating ongoing research in prompt engineering and security measures. The article also underscores the importance of structured software architectures, like the Model-View-Controller (MVC) design pattern, and best practices in software development to enhance code quality. Overall, these developments signify a transformative potential fo

In [38]:
# 요약 결과를 JSON 형식으로 저장합니다.
output_data = {
    "map_prompt": map_template,
    "reduce_template": reduce_template,
    "summary": answer["output_text"]
}
# 결과를 JSON 파일로 저장합니다.
with open("MapReduce_result.json", "w", encoding="utf-8") as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

print("JSON 파일로 저장되었습니다.")

JSON 파일로 저장되었습니다.


# 3.  Refine
##### 4개의 Chunk로 나뉘어진 문서를 요약하는 과정
##### 1번 chunk 요약 -> 1번 요약에 2번 chunk에 대한 내용 요약
##### 1+2번 요약에 3번 chunk를 더한 내용 요약
##### 1+2+3번 요약에 4번 chunk를 더한 내용 요약 -> 최종 결과
##### 문서의 맥락을 유지하며 긴 문서를 요약할 수 있는 방법이지만, Stuff 방법에 비하면 API 호출 횟수가 많음. 또한 LLM을 순차적으로 호출해야 하므로 전체 작업 시간이 길어짐

In [39]:
# llm을 사용하여 'refine' 유형의 요약 체인을 로드합니다.
chain = load_summarize_chain(llm, chain_type="refine")
# split_docs를 처리하기 위해 체인을 실행합니다.
chain.run(split_docs)

/tmp/ipykernel_1081/2162488473.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(split_docs)


The article "LLM Powered Autonomous Agents" by Lilian Weng, published on June 23, 2023, explores the architecture and functionality of autonomous agents powered by large language models (LLMs). It outlines three main components: 

1. **Planning** - Involves task decomposition and self-reflection to enhance decision-making.
2. **Memory** - Discusses different types of memory and the use of Maximum Inner Product Search (MIPS) for efficient retrieval.
3. **Tool Use** - Provides case studies, including a scientific discovery agent and generative agents simulation, along with proof-of-concept examples.

The article also addresses the challenges faced by these agents and includes citations and references for further reading.The article "LLM Powered Autonomous Agents" by Lilian Weng, published on June 23, 2023, explores the architecture and functionality of autonomous agents powered by large language models (LLMs). It outlines three main components:

1. **Planning** - Involves task decomposit

'The original summary remains relevant and does not require refinement based on the provided context about writing a Super Mario game in Python, pytest, and dataclasses. The focus of the original summary is on LLM-powered autonomous agents, their architecture, functionality, and various components such as planning, memory, and tool use, which are distinct from the context of game development and testing frameworks. Therefore, the original summary is retained as is.'

In [40]:
# 기본 요약을 위한 프롬프트 정의
prompt_template = """Write a concise summary of the following news article:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# Refine 프롬프트를 정의
refine_template = (
    "Your task is to produce a final summary.\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "Now, we have some additional context below that might help refine the summary.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given this new context, refine the original summary in 8 sentences or fewer, "
    "but only if the new context adds significant value. "
    "If the new context doesn't contribute meaningfully, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain.invoke({"input_documents": split_docs}, return_only_outputs=True)

The article "LLM Powered Autonomous Agents" by Lilian Weng, published on June 23, 2023, explores the architecture and functionality of autonomous agents powered by large language models (LLMs). It outlines three key components: planning, which includes task decomposition and self-reflection; memory, detailing types of memory and maximum inner product search (MIPS); and tool use, illustrated through case studies such as scientific discovery agents and generative agents simulations. The article also addresses the challenges faced by these agents and provides references for further reading.The article "LLM Powered Autonomous Agents" by Lilian Weng, published on June 23, 2023, explores the architecture and functionality of autonomous agents powered by large language models (LLMs). It outlines three key components: planning, which involves task decomposition into manageable subgoals and self-reflection for continuous improvement; memory, detailing types of memory and maximum inner product s

In [41]:
print(
    result["output_text"]
)  # 결과 딕셔너리에서 'output_text' 키에 해당하는 값을 출력합니다.

The original summary remains accurate and comprehensive without needing changes. The article "LLM Powered Autonomous Agents" by Lilian Weng, published on June 23, 2023, explores the architecture and functionality of autonomous agents powered by large language models (LLMs). It outlines three key components: planning, memory, and tool use, emphasizing agents' ability to call external APIs for enhanced capabilities. The ReAct framework integrates reasoning and acting, while the Reflexion framework enhances reasoning skills through dynamic memory and self-reflection. A significant aspect of the memory component is the long-term memory module, which records agents' experiences in natural language, facilitating inter-agent communication and context retrieval. The reflection mechanism synthesizes these memories into higher-level inferences, guiding future actions and optimizing decision-making. The article highlights challenges such as hallucination, limited context length, and the difficult

In [42]:
print("\n\n".join(result["intermediate_steps"][:3]))

The article "LLM Powered Autonomous Agents" by Lilian Weng, published on June 23, 2023, explores the architecture and functionality of autonomous agents powered by large language models (LLMs). It outlines three key components: planning, which includes task decomposition and self-reflection; memory, detailing types of memory and maximum inner product search (MIPS); and tool use, illustrated through case studies such as scientific discovery agents and generative agents simulations. The article also addresses the challenges faced by these agents and provides references for further reading.

The article "LLM Powered Autonomous Agents" by Lilian Weng, published on June 23, 2023, explores the architecture and functionality of autonomous agents powered by large language models (LLMs). It outlines three key components: planning, which involves task decomposition into manageable subgoals and self-reflection for continuous improvement; memory, detailing types of memory and maximum inner product

In [43]:
print(result["output_text"])

The original summary remains accurate and comprehensive without needing changes. The article "LLM Powered Autonomous Agents" by Lilian Weng, published on June 23, 2023, explores the architecture and functionality of autonomous agents powered by large language models (LLMs). It outlines three key components: planning, memory, and tool use, emphasizing agents' ability to call external APIs for enhanced capabilities. The ReAct framework integrates reasoning and acting, while the Reflexion framework enhances reasoning skills through dynamic memory and self-reflection. A significant aspect of the memory component is the long-term memory module, which records agents' experiences in natural language, facilitating inter-agent communication and context retrieval. The reflection mechanism synthesizes these memories into higher-level inferences, guiding future actions and optimizing decision-making. The article highlights challenges such as hallucination, limited context length, and the difficult

In [44]:
# 요약 결과를 JSON 형식으로 저장합니다.
output_data = {
    "question_template": prompt_template,
    "reduce_template": refine_template,
    "summary": answer["output_text"]
}
# 결과를 JSON 파일로 저장합니다.
with open("Refine_result.json", "w", encoding="utf-8") as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

print("JSON 파일로 저장되었습니다.")

JSON 파일로 저장되었습니다.
